<a href="https://colab.research.google.com/github/nicolapoggialini/thesis/blob/main/GNNs_examples/MPNN_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MPNN on MNIST

This notebook accompanies my blog on Graph Neural Networks and takes inspiration from PyTorch Geometric tutorial.

In [ ]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu111.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu111.html
!pip install torch-geometric
!pip install torch-cluster -f https://data.pyg.org/whl/torch-1.10.0+cu111.html
!pip install torch-spline-conv -f https://data.pyg.org/whl/torch-1.10.0+cu111.html

Looking in links: https://data.pyg.org/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 7.9 MB 5.3 MB/s 
Looking in links: https://data.pyg.org/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 3.5 MB 4.9 MB/s 
     |████████████████████████████████| 370 kB 5.3 MB/s 
     |████████████████████████████████| 482 kB 39.0 MB/s 
     |████████████████████████████████| 41 kB 619 kB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.3-py3-none-any.whl size=581968 sha256=c2cb2595de52264e3619c6366c66fdd870c38138794ae720e626e2d09ea4062d
  Stored in directory: /root/.cache/pip/wheels/c3/2a/58/87ce0508964d4def1aafb92750c4f3ac77038efd1b9a89dcf5
Successfully built torch-geometric
Looking in links: https://data.pyg.org/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 2.3 MB 5.4 MB/s 
Looking in links: https://data.pyg.org/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 747 kB 5.1 MB/s 


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.datasets import MNISTSuperpixels
from torch_geometric.loader import DataLoader
from torch_geometric.nn import NNConv, global_mean_pool, graclus, max_pool, max_pool_x
from torch_geometric.utils import normalized_cut

In [ ]:
transform = T.Cartesian(cat=False)
train_dataset = MNISTSuperpixels(".", True, transform=transform)
test_dataset = MNISTSuperpixels(".", False, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
d = train_dataset

Extracting ./raw/MNISTSuperpixels.zip
Processing...
Done!


In [ ]:
def normalized_cut_2d(edge_index, pos):
    row, col = edge_index
    edge_attr = torch.norm(pos[row] - pos[col], p=2, dim=1)
    return normalized_cut(edge_index, edge_attr, num_nodes=pos.size(0))


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        nn1 = nn.Sequential(nn.Linear(2, 25), nn.ReLU(),
                            nn.Linear(25, d.num_features * 32))
        self.conv1 = NNConv(d.num_features, 32, nn1, aggr='mean')

        nn2 = nn.Sequential(nn.Linear(2, 25), nn.ReLU(),
                            nn.Linear(25, 32 * 64))
        self.conv2 = NNConv(32, 64, nn2, aggr='mean')

        self.fc1 = torch.nn.Linear(64, 128)
        self.fc2 = torch.nn.Linear(128, d.num_classes)

    def forward(self, data):
        data.x = F.elu(self.conv1(data.x, data.edge_index, data.edge_attr))
        weight = normalized_cut_2d(data.edge_index, data.pos)
        cluster = graclus(data.edge_index, weight, data.x.size(0))
        data.edge_attr = None
        data = max_pool(cluster, data, transform=transform)

        data.x = F.elu(self.conv2(data.x, data.edge_index, data.edge_attr))
        weight = normalized_cut_2d(data.edge_index, data.pos)
        cluster = graclus(data.edge_index, weight, data.x.size(0))
        x, batch = max_pool_x(cluster, data.x, data.batch)

        x = global_mean_pool(x, batch)
        x = F.elu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        return F.log_softmax(self.fc2(x), dim=1)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


def train(epoch):
    model.train()

    if epoch == 16:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.001

    if epoch == 26:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.0001

    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        F.nll_loss(model(data), data.y).backward()
        optimizer.step()


def test():
    model.eval()
    correct = 0

    for data in test_loader:
        data = data.to(device)
        pred = model(data).max(1)[1]
        correct += pred.eq(data.y).sum().item()
    return correct / len(test_dataset)


for epoch in range(1, 31):
    train(epoch)
    test_acc = test()
    print(f'Epoch: {epoch:02d}, Test: {test_acc:.4f}')

Epoch: 01, Test: 0.6488
Epoch: 02, Test: 0.7352
Epoch: 03, Test: 0.7931
Epoch: 04, Test: 0.8408
Epoch: 05, Test: 0.8572
Epoch: 06, Test: 0.8607
Epoch: 07, Test: 0.8675
Epoch: 08, Test: 0.8744
Epoch: 09, Test: 0.8847
Epoch: 10, Test: 0.8830
Epoch: 11, Test: 0.9006
Epoch: 12, Test: 0.9007
Epoch: 13, Test: 0.8981
Epoch: 14, Test: 0.8736
Epoch: 15, Test: 0.9032
Epoch: 16, Test: 0.9290
Epoch: 17, Test: 0.9276
Epoch: 18, Test: 0.9264
Epoch: 19, Test: 0.9320
Epoch: 20, Test: 0.9314
Epoch: 21, Test: 0.9314
Epoch: 22, Test: 0.9318
Epoch: 23, Test: 0.9328
Epoch: 24, Test: 0.9318
Epoch: 25, Test: 0.9342
Epoch: 26, Test: 0.9369
Epoch: 27, Test: 0.9343
Epoch: 28, Test: 0.9347
Epoch: 29, Test: 0.9342
Epoch: 30, Test: 0.9355
